In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as bs
import mysql.connector
import time

In [2]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="job_scraping"
)

cursor = mydb.cursor()

# Create the table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Marketing_Jobs_Linkedin (
        JobId Varchar(20) PRIMARY KEY,
        Title VARCHAR(200),
        Company VARCHAR(100),
        Location VARCHAR(100),
        Apply VARCHAR(300),
        JobDescription TEXT,
        Date VARCHAR(50),
        Skills VARCHAR(400),
        Salary VARCHAR(100),
        CompanyLink VARCHAR(300)
    )
''')

In [3]:
# Define chrome_options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=./chromedriver.exe")

In [4]:
wd = webdriver.Chrome(options = chrome_options)
wd.get("https://linkedin.com/uas/login")
time.sleep(3)
            
username = wd.find_element(By.ID, "username")
username.send_keys("nguyenhuutienthanh23@gmail.com")

password = wd.find_element(By.ID, "password")
password.send_keys("Nhtt2002@")
            
wd.find_element(By.XPATH, "//button[@type = 'submit']").click()

In [5]:
driver = webdriver.Chrome(options = chrome_options)
driver.get("https://linkedin.com/uas/login")
time.sleep(3)
            
username = driver.find_element(By.ID, "username")
username.send_keys("nguyenhuutienthanh23@gmail.com")

password = driver.find_element(By.ID, "password")
password.send_keys("Nhtt2002@")
            
driver.find_element(By.XPATH, "//button[@type = 'submit']").click()

In [6]:
def linkedin_scraper(webpage, page_number):
    next_page = webpage + str(page_number)
    print(str(next_page))
    wd.get(next_page)
    
    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')
    
    for job in jobs:
        job_id = job.get("data-entity-urn").split(":")[3]
        # Check if the JobId already exists in the table
        select_query = "SELECT JobId FROM Marketing_Jobs_Linkedin WHERE JobId = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()
        
        if result is None:
            job_title = job.find('h3', class_='base-search-card__title').text.strip()
            job_company = job.find('h4', class_='base-search-card__subtitle').text.strip()
            job_location = job.find('span', class_='job-search-card__location').text.strip()
            job_link = job.find('a', class_='base-card__full-link')['href']
            posted_date_elem = job.find('time', class_='job-search-card__listdate')
            posted_date = posted_date_elem['datetime'] if posted_date_elem else ''
            company_link = job.find('a', class_='hidden-nested-link')['href']
            
            driver.get(job_link)
            time.sleep(3)
            job_sc = driver.page_source
            job_soup = bs(job_sc, "html.parser")
            salary_elem = job_soup.find("a", href = "#SALARY")
            salary = salary_elem.text.strip() if salary_elem else ""
            job_des_elem = job_soup.find("div", id = "job-details")
            job_des = job_des_elem.find("span").text.strip() if job_des_elem else ""
            
            list_skill = ""
            try:
                button = driver.find_element(By.XPATH, "//button[@class = 'jobs-unified-top-card__job-insight-text-button']")
                button.click()
                time.sleep(3)
                skill_list = []
                skill_src = driver.page_source
                skill_soup = bs(skill_src, "html.parser")
                skills = skill_soup.find_all("li", class_ = "job-details-skill-match-status-list__unmatched-skill")
                for skill in skills:
                    skill_list.append(skill.find("div").text.strip())

                list_skill = ", ".join(skill_list)
            except NoSuchElementException:
                list_skill = " "
                print("Button element not found")
            #Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO Marketing_Jobs_Linkedin (JobId, Title, Company, Location, Apply, JobDescription, Date, Skills, Salary, CompanyLink)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, job_company, job_location, job_link, job_des, posted_date, list_skill, salary, company_link)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
            
        else:
            print("Skipping duplicate JobId:", job_id)
        
    print("Data updated")
    
    if page_number < 1000:
        page_number += 25
        linkedin_scraper(webpage, page_number)
    else:
        cursor.close()
        mydb.close()


linkedin_scraper('https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Marketing&location=United%2BStates&locationId=&geoId=103644278&f_TPR=&f_E=2&start=', 0)

https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Marketing&location=United%2BStates&locationId=&geoId=103644278&f_TPR=&f_E=2&start=0
Skipping duplicate JobId: 3657181350
Skipping duplicate JobId: 3657176951
Skipping duplicate JobId: 3661103933
Skipping duplicate JobId: 3660944876
Skipping duplicate JobId: 3660032708
Skipping duplicate JobId: 3660767409
Skipping duplicate JobId: 3657738348
Skipping duplicate JobId: 3664073993
Skipping duplicate JobId: 3661201794
Skipping duplicate JobId: 3662352967
Skipping duplicate JobId: 3657057277
Skipping duplicate JobId: 3659602222
Skipping duplicate JobId: 3663547366
Skipping duplicate JobId: 3660032220
Skipping duplicate JobId: 3662369213
Skipping duplicate JobId: 3663284843
Skipping duplicate JobId: 3662369229
Skipping duplicate JobId: 3660724129
Skipping duplicate JobId: 3662729861
Skipping duplicate JobId: 3658310382
Skipping duplicate JobId: 3658026216
Skipping duplicate JobId: 3657039631
Skipping duplicate Job